# Extract Images from the PDF.

In [ ]:
import fitz  # PyMuPDF for working with PDF files.
from PIL import Image  # Python Imaging Library for handling images.
import io  # Core tools for working with I/O operations.

def is_valid_image(image_bytes, threshold=0.95):
    """
    Check if the image is valid (not mostly black).
    """
    with Image.open(io.BytesIO(image_bytes)) as img:  # Open the image from bytes.
        img = img.convert("RGB")  # Convert the image to RGB.
        pixels = img.load()  # Load the pixel data.
        width, height = img.size  # Get the dimensions of the image.
        black_pixel_count = 0  # Initialize the count for black pixels.
        total_pixels = width * height  # Calculate the total number of pixels.
        
        # Count black pixels.
        for x in range(width):  # Loop over the width of the image.
            for y in range(height):  # Loop over the height of the image.
                r, g, b = pixels[x, y]  # Get the RGB values of the pixel.
                if r == 0 and g == 0 and b == 0:  # Check if the pixel is black.
                    black_pixel_count += 1  # Increment the black pixel count.

        # Calculate the percentage of black pixels.
        black_pixel_percentage = black_pixel_count / total_pixels  # Compute the percentage.
        return black_pixel_percentage < threshold  # Return if the black pixel percentage is below the threshold.

def save_images_from_pdf(pdf_path, output_folder):
    # Open the PDF file.
    pdf_document = fitz.open(pdf_path)  # Open the PDF file using PyMuPDF.
    
    # Iterate through each page in the PDF.
    for page_num in range(len(pdf_document)):  # Loop over each page in the PDF.
        page = pdf_document.load_page(page_num)  # Load the specific page.
        image_list = page.get_images(full=True)  # Get the list of images on the page.
        
        for img_index, img in enumerate(image_list):  # Loop over each image in the list.
            xref = img[0]  # Get the reference number for the image.
            base_image = pdf_document.extract_image(xref)  # Extract the image from the PDF.
            image_bytes = base_image["image"]  # Get the image bytes.
            
            if is_valid_image(image_bytes):  # Check if the image is valid.
                image_filename = f"{output_folder}/page_{page_num + 1}_img_{img_index + 1}.png"  # Define the filename for the image.
                
                with open(image_filename, "wb") as img_file:  # Open the file in write-binary mode.
                    img_file.write(image_bytes)  # Write the image bytes to the file.
                print(f"Saved image {image_filename}")  # Print a message indicating the image was saved.
            else:
                print(f"Skipped invalid image from page {page_num + 1}, image {img_index + 1}.")  # Print a message indicating the image was skipped.
    
    print("Finished extracting images.")  # Print a message indicating the process is finished.
    pdf_document.close()  # Close the PDF document.

# Example usage.
pdf_path = "Animal_Pictures.pdf"  # Path for the PDF file defined here.
output_folder = "Output_Images"  # Define the output folder for saving images.

# Ensure the output folder exists.
import os  # Import the OS module for operating system dependent functionality.
if not os.path.exists(output_folder):  # Check if the output folder does not exist.
    os.makedirs(output_folder)  # Create the output folder if it does not exist.

save_images_from_pdf(pdf_path, output_folder)  # Call the function to save images from the PDF.

# Read the Annotations.

In [ ]:
# Create a .json file as an example.

import json

annotations = {
    "annotations": [
        {"type": "image", "page": 1, "coordinates": {"x": 0, "y": 0, "width": 300, "height": 200}, "label": "cow"},
        {"type": "image", "page": 1, "coordinates": {"x": 0, "y": 0, "width": 300, "height": 200}, "label": "cow"},
        {"type": "image", "page": 3, "coordinates": {"x": 0, "y": 0, "width": 300, "height": 200}, "label": "goat"},
        {"type": "image", "page": 3, "coordinates": {"x": 0, "y": 0, "width": 300, "height": 200}, "label": "goat"},
        {"type": "image", "page": 5, "coordinates": {"x": 0, "y": 0, "width": 300, "height": 200}, "label": "lama"},
        {"type": "image", "page": 5, "coordinates": {"x": 0, "y": 0, "width": 300, "height": 200}, "label": "sheep"},
        {"type": "image", "page": 7, "coordinates": {"x": 0, "y": 0, "width": 300, "height": 200}, "label": "chicken"},
        {"type": "image", "page": 7, "coordinates": {"x": 0, "y": 0, "width": 300, "height": 200}, "label": "chicken"},
        {"type": "image", "page": 9, "coordinates": {"x": 0, "y": 0, "width": 300, "height": 200}, "label": "horse"},
        {"type": "image", "page": 9, "coordinates": {"x": 0, "y": 0, "width": 300, "height": 200}, "label": "horse"},
        {"type": "image", "page": 11, "coordinates": {"x": 0, "y": 0, "width": 300, "height": 200}, "label": "pig"},
        {"type": "image", "page": 11, "coordinates": {"x": 0, "y": 0, "width": 300, "height": 200}, "label": "pig"},
        {"type": "image", "page": 13, "coordinates": {"x": 0, "y": 0, "width": 300, "height": 200}, "label": "cow"},
        {"type": "image", "page": 13, "coordinates": {"x": 0, "y": 0, "width": 300, "height": 200}, "label": "chicken"},
        {"type": "image", "page": 15, "coordinates": {"x": 0, "y": 0, "width": 300, "height": 200}, "label": "sheep"},
        {"type": "image", "page": 15, "coordinates": {"x": 0, "y": 0, "width": 300, "height": 200}, "label": "horse"},
        {"type": "image", "page": 17, "coordinates": {"x": 0, "y": 0, "width": 300, "height": 200}, "label": "pig"},
        {"type": "image", "page": 17, "coordinates": {"x": 0, "y": 0, "width": 300, "height": 200}, "label": "sheep"}
    ]
}

with open('./annotations.json', 'w') as json_file:
    json.dump(annotations, json_file, indent=4)

print("New annotations JSON file has been created.")


In [ ]:
import json

# Load the JSON file
with open('./annotations.json', 'r') as f:
    annotations = json.load(f)

print(annotations)

# Prepare the Dataset.

In [ ]:
output_folder = './Output_Images'

In [ ]:
from PIL import Image  # Import the Image class from the PIL (Pillow) library to handle image operations.
import numpy as np  # Import NumPy for array manipulations.

def load_images_and_labels(output_folder, annotations):
    length_of_annotations = len(annotations['annotations'])  # Determine the number of annotations provided.
    images = []  # Initialize an empty list to store image arrays.
    labels = []  # Initialize an empty list to store corresponding labels.
    image_paths = []  # Initialize an empty list to keep track of image paths processed.

    for annotation in annotations['annotations']:  # Iterate over each annotation in the annotations list.
        page = annotation['page']  # Extract the page number from the annotation.
        label = annotation['label']  # Extract the label from the annotation.
        x, y, width, height = annotation['coordinates'].values()  # Extract the bounding box coordinates from the annotation.

        for i in range(length_of_annotations):  # Loop through all possible image files based on the length of annotations.
            image_path = f"{output_folder}/page_{page}_img_{i}.png"  # Construct the path to the image file.
            if os.path.exists(image_path):  # Check if the image file exists at the specified path.
                if image_path not in image_paths:  # Ensure the image path has not been processed yet.
                    print("image_path", image_path)  # Print the image path for debugging or logging purposes.
                    with Image.open(image_path) as img:  # Open the image file.
                        img = img.convert('RGB')  # Convert the image to RGB mode to ensure it's in a standard format.
                        cropped_img = img.crop((x, y, x + width, y + height))  # Crop the image using the coordinates provided in the annotation.
                        img_array = np.array(cropped_img)  # Convert the cropped image to a NumPy array.
                        images.append(img_array)  # Append the image array to the images list.
                        labels.append(label)  # Append the corresponding label to the labels list.
                        image_paths.append(image_path)  # Add the image path to the list of processed paths.

    return np.array(images), np.array(labels)  # Convert the lists to NumPy arrays and return them.

images, labels = load_images_and_labels(output_folder, annotations)  # Call the function with the specified output folder and annotations.

print(images.shape, labels)  # Print the shape of the images array and the labels list to verify the results.

# Train a Classifier Using PyTorch.

In [ ]:
import torch  # Import PyTorch library.
import torch.nn as nn  # Import neural network module from PyTorch.
import torch.optim as optim  # Import optimization module from PyTorch.
from torch.utils.data import DataLoader, Dataset  # Import data handling utilities from PyTorch.
from sklearn.preprocessing import LabelEncoder  # Import LabelEncoder from scikit-learn.
from sklearn.model_selection import train_test_split  # Import train_test_split from scikit-learn.
import torchvision.transforms as transforms  # Import transforms from torchvision for image preprocessing.

# Define the dataset class.
class AnimalDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images  # Assign images to the class variable.
        self.labels = labels  # Assign labels to the class variable.
        self.transform = transform  # Assign transform to the class variable.
    
    def __len__(self):
        return len(self.images)  # Return the total number of samples in the dataset.
    
    def __getitem__(self, idx):
        image = self.images[idx]  # Get the image at the specified index.
        label = self.labels[idx]  # Get the label at the specified index.
        
        if self.transform:  # If transform is provided.
            image = self.transform(image)  # Apply the transform to the image.
        
        return image, label  # Return the image and its label.

# Preprocess images and labels.
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors.
    transforms.Resize((64, 64)),  # Resize images to 64x64 pixels.
    transforms.Normalize((0.5,), (0.5,))  # Normalize images to have mean 0.5 and std 0.5.
])

label_encoder = LabelEncoder()  # Initialize the LabelEncoder.
labels_encoded = label_encoder.fit_transform(labels)  # Encode the string labels as integers.

# Split the dataset into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(images, labels_encoded, test_size=0.2, random_state=42)  # 80% training, 20% testing.

train_dataset = AnimalDataset(X_train, y_train, transform=transform)  # Create the training dataset.
test_dataset = AnimalDataset(X_test, y_test, transform=transform)  # Create the testing dataset.

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)  # Create the training data loader.
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)  # Create the testing data loader.

# Define a simple CNN model.
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()  # Initialize the parent class.
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)  # First convolutional layer.
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)  # Second convolutional layer.
        self.fc1 = nn.Linear(32 * 16 * 16, 128)  # First fully connected layer.
        self.fc2 = nn.Linear(128, len(label_encoder.classes_))  # Second fully connected layer.
    
    def forward(self, x):
        x = torch.relu(self.conv1(x))  # Apply ReLU activation after the first convolutional layer.
        x = torch.max_pool2d(x, 2)  # Apply max pooling with a kernel size of 2.
        x = torch.relu(self.conv2(x))  # Apply ReLU activation after the second convolutional layer.
        x = torch.max_pool2d(x, 2)  # Apply max pooling with a kernel size of 2.
        x = x.view(x.size(0), -1)  # Flatten the tensor.
        x = torch.relu(self.fc1(x))  # Apply ReLU activation after the first fully connected layer.
        x = self.fc2(x)  # Apply the second fully connected layer.
        return x  # Return the output.

model = SimpleCNN()  # Instantiate the model.
criterion = nn.CrossEntropyLoss()  # Define the loss function.
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Define the optimizer.

# Training loop.
num_epochs = 1000  # Define the number of epochs for training.
for epoch in range(num_epochs):  # Loop over the number of epochs.
    model.train()  # Set the model to training mode.
    running_loss = 0.0  # Initialize the running loss.
    for images, labels in train_loader:  # Loop over the data in the dataloader.
        optimizer.zero_grad()  # Zero the gradients.
        outputs = model(images)  # Forward pass.
        loss = criterion(outputs, labels)  # Compute the loss.
        loss.backward()  # Backward pass.
        optimizer.step()  # Update the parameters.
        running_loss += loss.item()  # Accumulate the loss.
    
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")  # Print the average loss for the epoch.

# Evaluate the model.
model.eval()  # Set the model to evaluation mode.
correct = 0  # Initialize the number of correct predictions.
total = 0  # Initialize the total number of samples.
with torch.no_grad():  # Disable gradient calculation.
    for images, labels in test_loader:  # Loop over the data in the test loader.
        outputs = model(images)  # Forward pass.
        _, predicted = torch.max(outputs, 1)  # Get the predicted class.
        total += labels.size(0)  # Update the total number of samples.
        correct += (predicted == labels).sum().item()  # Update the number of correct predictions.

print(f'Accuracy: {100 * correct / total}%')  # Print the accuracy.

# Train using Uncertainty Sampling.

In [ ]:
import torch  # Importing PyTorch library.
import torch.nn as nn  # Importing the neural network module from PyTorch.
import torch.optim as optim  # Importing the optimization module from PyTorch.
from torch.utils.data import DataLoader, Dataset, Subset  # Importing data handling utilities from PyTorch.
from sklearn.preprocessing import LabelEncoder  # Importing LabelEncoder from scikit-learn.
from sklearn.model_selection import train_test_split  # Importing train_test_split from scikit-learn.
import torchvision.transforms as transforms  # Importing transforms from torchvision for image preprocessing.
import numpy as np  # Importing NumPy for numerical operations.

# Define the dataset class.
class AnimalDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images  # Assigning images to the class variable.
        self.labels = labels  # Assigning labels to the class variable.
        self.transform = transform  # Assigning transform to the class variable.
    
    def __len__(self):
        return len(self.images)  # Returning the total number of samples in the dataset.
    
    def __getitem__(self, idx):
        image = self.images[idx]  # Getting the image at the specified index.
        label = self.labels[idx]  # Getting the label at the specified index.
        
        if self.transform:  # If transform is provided.
            image = self.transform(image)  # Apply the transform to the image.
        
        return image, label  # Return the image and its label.

# Preprocess images and labels.
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors.
    transforms.Resize((64, 64)),  # Resize images to 64x64 pixels.
    transforms.Normalize((0.5,), (0.5,))  # Normalize images to have mean 0.5 and std 0.5.
])

label_encoder = LabelEncoder()  # Initialize the LabelEncoder.
labels_encoded = label_encoder.fit_transform(labels)  # Encode the string labels as integers.

X_train, X_test, y_train, y_test = train_test_split(images, labels_encoded, test_size=0.2, random_state=42)  # Split the dataset into training and testing sets.

train_dataset = AnimalDataset(X_train, y_train, transform=transform)  # Create the training dataset.
test_dataset = AnimalDataset(X_test, y_test, transform=transform)  # Create the testing dataset.

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)  # Create the training data loader.
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)  # Create the testing data loader.

# Define a simple CNN model.
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()  # Initialize the parent class.
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)  # First convolutional layer.
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)  # Second convolutional layer.
        self.fc1 = nn.Linear(32 * 16 * 16, 128)  # First fully connected layer.
        self.fc2 = nn.Linear(128, len(label_encoder.classes_))  # Second fully connected layer.
    
    def forward(self, x):
        x = torch.relu(self.conv1(x))  # Apply ReLU activation after the first convolutional layer.
        x = torch.max_pool2d(x, 2)  # Apply max pooling with a kernel size of 2.
        x = torch.relu(self.conv2(x))  # Apply ReLU activation after the second convolutional layer.
        x = torch.max_pool2d(x, 2)  # Apply max pooling with a kernel size of 2.
        x = x.view(x.size(0), -1)  # Flatten the tensor.
        x = torch.relu(self.fc1(x))  # Apply ReLU activation after the first fully connected layer.
        x = self.fc2(x)  # Apply the second fully connected layer.
        return x  # Return the output.

model = SimpleCNN()  # Instantiate the model.
criterion = nn.CrossEntropyLoss()  # Define the loss function.
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Define the optimizer.

# Training function.
def train_model(model, dataloader, criterion, optimizer, num_epochs=5):
    model.train()  # Set the model to training mode.
    for epoch in range(num_epochs):  # Loop over the number of epochs.
        running_loss = 0.0  # Initialize the running loss.
        for images, labels in dataloader:  # Loop over the data in the dataloader.
            optimizer.zero_grad()  # Zero the gradients.
            outputs = model(images)  # Forward pass.
            loss = criterion(outputs, labels)  # Compute the loss.
            loss.backward()  # Backward pass.
            optimizer.step()  # Update the parameters.
            running_loss += loss.item()  # Accumulate the loss.
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}')  # Print the average loss for the epoch.

# Uncertainty sampling function.
def get_uncertain_samples(model, dataloader, k):
    model.eval()  # Set the model to evaluation mode.
    uncertainties = []  # Initialize a list to store uncertainties.
    with torch.no_grad():  # Disable gradient calculation.
        for images, _ in dataloader:  # Loop over the data in the dataloader.
            outputs = model(images)  # Forward pass.
            probabilities = torch.nn.functional.softmax(outputs, dim=1)  # Apply softmax to get probabilities.
            entropy = -torch.sum(probabilities * torch.log(probabilities + 1e-10), dim=1)  # Compute the entropy for each sample.
            uncertainties.append(entropy)  # Append the entropy to the list.
    uncertainties = torch.cat(uncertainties)  # Concatenate the list of uncertainties.
    _, indices = torch.topk(uncertainties, k)  # Get the indices of the top k uncertainties.
    return indices  # Return the indices.

# Active Learning Loop.
num_initial_samples = 5  # Number of samples to start training with.
num_queries = 2  # Number of samples to query in each iteration.
num_iterations = 1000  # Number of active learning iterations.

# Initial training set.
initial_indices = np.random.choice(len(X_train), num_initial_samples, replace=False)  # Randomly select initial samples.
train_dataset = Subset(train_dataset, initial_indices)  # Create a subset of the training dataset with the initial samples.
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)  # Create a data loader for the initial training dataset.

for iteration in range(num_iterations):  # Loop over the number of iterations.
    print(f'Iteration {iteration+1}/{num_iterations}')  # Print the current iteration.
    train_model(model, train_loader, criterion, optimizer, num_epochs=1)  # Train the model for one epoch.

    # Get the most uncertain samples.
    remaining_indices = list(set(range(len(X_train))) - set(initial_indices))  # Get the remaining samples that are not in the training set.
    if len(remaining_indices) == 0:  # If there are no remaining samples.
        break  # Exit the loop.
    remaining_dataset = Subset(AnimalDataset(X_train, y_train, transform=transform), remaining_indices)  # Create a subset of the remaining samples.
    remaining_loader = DataLoader(remaining_dataset, batch_size=16, shuffle=False)  # Create a data loader for the remaining samples.
    uncertain_indices = get_uncertain_samples(model, remaining_loader, min(num_queries, len(remaining_indices)))  # Get the indices of the most uncertain samples.

    # Add the uncertain samples to the training set.
    initial_indices = np.concatenate([initial_indices, uncertain_indices])  # Add the uncertain samples to the initial indices.
    train_dataset = Subset(AnimalDataset(X_train, y_train, transform=transform), initial_indices)  # Create a new subset of the training dataset.
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)  # Create a new data loader for the updated training dataset.

# Final model evaluation.
model.eval()  # Set the model to evaluation mode.
correct = 0  # Initialize the number of correct predictions.
total = 0  # Initialize the total number of samples.
with torch.no_grad():  # Disable gradient calculation.
    for images, labels in test_loader:  # Loop over the data in the test loader.
        outputs = model(images)  # Forward pass.
        _, predicted = torch.max(outputs, 1)  # Get the predicted class.
        total += labels.size(0)  # Update the total number of samples.
        correct += (predicted == labels).sum().item()  # Update the number of correct predictions.

print(f'Accuracy: {100 * correct / total}%')  # Print the accuracy.